# Camera Simulation with SPIDS

This notebook demonstrates how to simulate general camera systems using SPIDS's multi-instrument framework.

## Key Features
- Automatic propagation regime selection based on Fresnel number
- Thin lens equation calculations (image distance, magnification)
- Depth of field (DOF) computation
- Defocus aberration modeling
- Realistic sensor noise simulation

## 1. Setup and Import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from prism.core.instruments import Camera, CameraConfig, create_instrument


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Standard Camera Configuration

Let's configure a standard 50mm f/1.4 camera, a popular choice for general photography.

In [ ]:
# Configure a 50mm f/1.4 camera
config_50mm = CameraConfig(
    focal_length=50e-3,  # 50mm
    f_number=1.4,  # f/1.4 (fast lens)
    sensor_size=(36e-3, 24e-3),  # Full frame sensor
    object_distance=2.0,  # 2 meters
    wavelength=550e-9,  # Green light
    n_pixels=512,
    pixel_size=6.5e-6,  # Typical sensor pixel
)

# Create camera
camera_50mm = Camera(config_50mm)

# Display camera information
info = camera_50mm.get_info()
print("50mm f/1.4 Camera Configuration:")
print("=" * 40)
for key, value in info.items():
    print(f"{key:<20}: {value}")

# Calculate optical parameters
aperture_diameter = camera_50mm.focal_length / camera_50mm.f_number
image_distance = camera_50mm.calculate_image_distance()
magnification = camera_50mm.calculate_magnification()
near_dof, far_dof = camera_50mm.calculate_depth_of_field()

# Get fresnel number from info dict
fresnel_number = info.get("fresnel_number", 0)

print("\nOptical Parameters:")
print(f"Aperture diameter: {aperture_diameter * 1e3:.1f} mm")
print(f"Image distance: {image_distance * 1e3:.2f} mm")
print(f"Magnification: {magnification:.4f}x")
print(f"Depth of field: {near_dof:.2f} m to {far_dof:.2f} m")
print(f"DOF range: {far_dof - near_dof:.2f} m")
print(f"Fresnel number: {fresnel_number:.4f}")
print(f"Propagation regime: {'Far-field' if fresnel_number < 0.1 else 'Near-field'}")
print(f"Resolution limit: {camera_50mm.resolution_limit * 1e6:.1f} um")

## 3. Point Spread Function (PSF) Analysis

The PSF shows how the camera images a point source.

In [ ]:
# Compute in-focus PSF
psf_focus = camera_50mm.compute_psf(defocus=0)

# Compute defocused PSFs
defocus_amounts = [0, 0.5e-3, 1e-3, 2e-3]  # mm of defocus
psfs = {}
for defocus in defocus_amounts:
    psfs[defocus * 1e3] = camera_50mm.compute_psf(defocus=defocus).cpu().numpy()

# Visualize PSFs
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, defocus_mm in enumerate(psfs.keys()):
    psf = psfs[defocus_mm]

    # 2D PSF
    ax = axes[0, idx]
    center = psf.shape[0] // 2
    zoom = 50
    im = ax.imshow(
        psf[center - zoom : center + zoom, center - zoom : center + zoom],
        cmap="hot",
        interpolation="nearest",
    )
    ax.set_title(f"Defocus = {defocus_mm:.1f} mm")
    ax.axis("off")

    # Cross-section
    ax = axes[1, idx]
    ax.plot(psf[center, center - zoom : center + zoom], "b-", linewidth=2)
    ax.set_title("Cross-section")
    ax.set_xlabel("Pixels")
    ax.set_ylabel("Intensity")
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0, 1.1])

plt.suptitle("Camera PSF with Varying Defocus", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Calculate and display PSF widths
print("\nPSF Analysis:")
print("=" * 50)
for defocus_mm, psf in psfs.items():
    center_line = psf[psf.shape[0] // 2, :]
    half_max = 0.5 * psf.max()
    indices = np.where(center_line >= half_max)[0]
    if len(indices) > 0:
        fwhm_pixels = indices[-1] - indices[0]
        fwhm_um = fwhm_pixels * config_50mm.pixel_size * 1e6
        print(f"Defocus {defocus_mm:4.1f} mm: FWHM = {fwhm_pixels:3d} pixels ({fwhm_um:6.1f} μm)")

## 4. Depth of Field Demonstration

Let's visualize how aperture size affects depth of field.

In [ ]:
# Compare different f-numbers
f_numbers = [1.4, 2.8, 5.6, 11]
focal_distances = np.linspace(0.5, 10, 100)

# Calculate DOF for each f-number
dof_data = {}
for f_num in f_numbers:
    near_limits = []
    far_limits = []

    for focus_dist in focal_distances:
        # Temporary config for calculation
        config = CameraConfig(
            focal_length=50e-3,
            f_number=f_num,
            object_distance=focus_dist,
            focus_distance=focus_dist,
            wavelength=550e-9,
            n_pixels=256,
            pixel_size=6.5e-6,
        )
        cam = Camera(config)
        near, far = cam.calculate_depth_of_field(coc_limit=30e-6)  # 30μm circle of confusion
        near_limits.append(near)
        far_limits.append(min(far, 100))  # Cap at 100m for visualization

    dof_data[f_num] = {"near": np.array(near_limits), "far": np.array(far_limits)}

# Visualize DOF
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
colors = ["blue", "green", "orange", "red"]

for idx, (f_num, color) in enumerate(zip(f_numbers, colors)):
    ax = axes[idx // 2, idx % 2]

    # Plot DOF region
    ax.fill_between(
        focal_distances,
        dof_data[f_num]["near"],
        dof_data[f_num]["far"],
        alpha=0.3,
        color=color,
        label="In Focus Range",
    )
    ax.plot(focal_distances, focal_distances, "k--", linewidth=1, label="Focus Distance")
    ax.plot(focal_distances, dof_data[f_num]["near"], color=color, linewidth=2, label="Near Limit")
    ax.plot(focal_distances, dof_data[f_num]["far"], color=color, linewidth=2, label="Far Limit")

    ax.set_title(f"f/{f_num} - Aperture Diameter: {50 / f_num:.1f}mm")
    ax.set_xlabel("Focus Distance (m)")
    ax.set_ylabel("Distance (m)")
    ax.set_ylim([0, 20])
    ax.grid(True, alpha=0.3)
    ax.legend(loc="upper left")

    # Add DOF range annotation at specific focus distance
    focus_2m_idx = np.argmin(np.abs(focal_distances - 2.0))
    near_2m = dof_data[f_num]["near"][focus_2m_idx]
    far_2m = dof_data[f_num]["far"][focus_2m_idx]
    dof_range = far_2m - near_2m
    ax.annotate(
        f"DOF at 2m: {dof_range:.2f}m",
        xy=(2, (near_2m + far_2m) / 2),
        xytext=(3, (near_2m + far_2m) / 2 + 2),
        arrowprops=dict(arrowstyle="->", color="black", alpha=0.5),
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.5),
    )

plt.suptitle("Depth of Field vs. Aperture (50mm lens)", fontsize=14)
plt.tight_layout()
plt.show()

# Print DOF comparison at 2m focus
print("\nDOF Comparison at 2m Focus Distance:")
print("=" * 60)
print(f"{'f-number':<10} {'Aperture (mm)':<15} {'Near (m)':<10} {'Far (m)':<10} {'Range (m)':<10}")
print("-" * 60)
for f_num in f_numbers:
    focus_2m_idx = np.argmin(np.abs(focal_distances - 2.0))
    near = dof_data[f_num]["near"][focus_2m_idx]
    far = dof_data[f_num]["far"][focus_2m_idx]
    print(f"f/{f_num:<8} {50 / f_num:<15.1f} {near:<10.2f} {far:<10.2f} {far - near:<10.2f}")

## 5. Image Formation Simulation

Let's simulate actual image formation through the camera system.

In [ ]:
# Create a test scene with multiple point sources
def create_star_field(size, n_stars=20):
    """Create a field of point sources (stars)."""
    scene = torch.zeros((size, size), dtype=torch.complex64)
    np.random.seed(42)  # For reproducibility

    for _ in range(n_stars):
        x = np.random.randint(50, size - 50)
        y = np.random.randint(50, size - 50)
        brightness = np.random.uniform(0.3, 1.0)
        scene[y, x] = brightness

    return scene


# Create test scene
star_field = create_star_field(512, n_stars=15)

# Image the scene with different camera settings
camera_configs = [
    {"f_number": 1.4, "label": "f/1.4 (Wide Open)"},
    {"f_number": 2.8, "label": "f/2.8"},
    {"f_number": 5.6, "label": "f/5.6"},
    {"f_number": 8.0, "label": "f/8 (Sharp)"},
]

images = {}
for cfg in camera_configs:
    config = CameraConfig(
        focal_length=50e-3,
        f_number=cfg["f_number"],
        object_distance=float("inf"),  # Stars at infinity
        wavelength=550e-9,
        n_pixels=512,
        pixel_size=6.5e-6,
    )
    camera = Camera(config)

    # Image without noise
    image = camera.forward(star_field, add_noise=False)
    images[cfg["label"]] = image.cpu().numpy()

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original scene
ax = axes[0, 0]
ax.imshow(torch.abs(star_field).numpy(), cmap="hot", vmin=0, vmax=1)
ax.set_title("Original Scene (Point Sources)")
ax.axis("off")

# Different f-numbers
for idx, (label, image) in enumerate(images.items()):
    row = (idx + 1) // 3
    col = (idx + 1) % 3
    ax = axes[row, col]

    # Normalize for display
    im = ax.imshow(image, cmap="hot", vmin=0, vmax=image.max())
    ax.set_title(label)
    ax.axis("off")

# Zoom comparison in last subplot
ax = axes[1, 2]
# Pick a region with a star
y, x = 256, 256
zoom_size = 30
for label, image in images.items():
    profile = image[y, x - zoom_size : x + zoom_size]
    f_num = float(label.split("/")[1].split()[0])
    ax.plot(profile / profile.max(), label=label, linewidth=2)
ax.set_title("PSF Comparison (Cross-sections)")
ax.set_xlabel("Pixels")
ax.set_ylabel("Normalized Intensity")
ax.legend()
ax.grid(True, alpha=0.3)

plt.suptitle("Image Formation with Different Apertures", fontsize=14)
plt.tight_layout()
plt.show()

# Print diffraction-limited spot sizes
print("\nDiffraction-Limited Spot Sizes:")
print("=" * 50)
for cfg in camera_configs:
    spot_size = 2.44 * 550e-9 * cfg["f_number"] * 1e6  # in micrometers
    spot_pixels = spot_size / 6.5  # 6.5 μm pixels
    print(f"{cfg['label']:<20}: {spot_size:6.2f} μm ({spot_pixels:.1f} pixels)")

## 6. Sensor Noise Modeling

Real cameras have various noise sources: shot noise, read noise, and dark current.

In [ ]:
# Create a test pattern with varying intensities
def create_gradient_target(size):
    """Create a gradient test pattern."""
    pattern = torch.zeros((size, size), dtype=torch.complex64)

    # Horizontal gradient
    for i in range(size):
        intensity = i / size
        pattern[:, i] = intensity

    # Add some features
    pattern[100:150, 100:150] = 1.0  # Bright square
    pattern[300:350, 300:350] = 0.5  # Medium square
    pattern[200:250, 100:400] = 0.3  # Horizontal bar

    return pattern


# Create test pattern
test_pattern = create_gradient_target(512)

# Configure camera
config = CameraConfig(
    focal_length=50e-3,
    f_number=2.8,
    object_distance=1.0,
    wavelength=550e-9,
    n_pixels=512,
    pixel_size=6.5e-6,
)
camera = Camera(config)

# Image with and without noise
image_clean = camera.forward(test_pattern, add_noise=False)
image_noisy1 = camera.forward(test_pattern, add_noise=True)
image_noisy2 = camera.forward(test_pattern, add_noise=True)
image_noisy3 = camera.forward(test_pattern, add_noise=True)

# Calculate noise statistics from multiple samples
n_samples = 20
noisy_stack = torch.stack([camera.forward(test_pattern, add_noise=True) for _ in range(n_samples)])
noise_mean = torch.mean(noisy_stack, dim=0)
noise_std = torch.std(noisy_stack, dim=0)

# Visualize noise effects
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Clean image
ax = axes[0, 0]
im = ax.imshow(image_clean.cpu().numpy(), cmap="gray", vmin=0, vmax=1)
ax.set_title("Clean Image (No Noise)")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046)

# First noisy realization
ax = axes[0, 1]
im = ax.imshow(image_noisy1.cpu().numpy(), cmap="gray", vmin=0, vmax=1)
ax.set_title("Noisy Image (Sample 1)")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046)

# Second noisy realization
ax = axes[0, 2]
im = ax.imshow(image_noisy2.cpu().numpy(), cmap="gray", vmin=0, vmax=1)
ax.set_title("Noisy Image (Sample 2)")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046)

# Mean of noisy images
ax = axes[1, 0]
im = ax.imshow(noise_mean.cpu().numpy(), cmap="gray", vmin=0, vmax=1)
ax.set_title(f"Mean of {n_samples} Noisy Images")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046)

# Standard deviation map
ax = axes[1, 1]
im = ax.imshow(noise_std.cpu().numpy(), cmap="hot", vmin=0, vmax=0.05)
ax.set_title("Noise Standard Deviation")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046)

# SNR map
ax = axes[1, 2]
snr = image_clean / (noise_std + 1e-10)
im = ax.imshow(snr.cpu().numpy(), cmap="viridis", vmin=0, vmax=100)
ax.set_title("Signal-to-Noise Ratio")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046, label="SNR")

# Noise histogram
ax = axes[2, 0]
noise = (image_noisy1 - image_clean).cpu().numpy().flatten()
ax.hist(noise, bins=50, alpha=0.7, color="blue", density=True)
ax.axvline(x=0, color="red", linestyle="--", label="Zero mean")
ax.set_title("Noise Distribution")
ax.set_xlabel("Noise Value")
ax.set_ylabel("Probability Density")
ax.legend()
ax.grid(True, alpha=0.3)

# Intensity vs. noise relationship
ax = axes[2, 1]
intensity_bins = np.linspace(0, 1, 20)
mean_noise_std = []
for i in range(len(intensity_bins) - 1):
    mask = (image_clean.cpu() >= intensity_bins[i]) & (image_clean.cpu() < intensity_bins[i + 1])
    if mask.any():
        mean_noise_std.append(noise_std.cpu()[mask].mean().item())
    else:
        mean_noise_std.append(0)

ax.plot(intensity_bins[:-1], mean_noise_std, "bo-", linewidth=2, markersize=8)
ax.set_title("Noise vs. Signal Intensity")
ax.set_xlabel("Signal Intensity")
ax.set_ylabel("Noise Std Dev")
ax.grid(True, alpha=0.3)

# Line profiles comparison
ax = axes[2, 2]
y_line = 256
ax.plot(image_clean.cpu()[y_line, :], "b-", label="Clean", linewidth=2)
ax.plot(image_noisy1.cpu()[y_line, :], "r-", alpha=0.5, label="Noisy 1")
ax.plot(noise_mean.cpu()[y_line, :], "g--", label=f"Mean ({n_samples} samples)", linewidth=2)
ax.set_title(f"Line Profile at Y={y_line}")
ax.set_xlabel("X Position (pixels)")
ax.set_ylabel("Intensity")
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim([-0.1, 1.1])

plt.suptitle("Sensor Noise Analysis", fontsize=14)
plt.tight_layout()
plt.show()

# Print noise statistics
print("\nNoise Statistics:")
print("=" * 50)
print(f"Mean noise: {noise.mean():.6f}")
print(f"Std noise: {noise.std():.6f}")
print(f"Min noise: {noise.min():.6f}")
print(f"Max noise: {noise.max():.6f}")
print("\nSNR Statistics:")
print(f"Mean SNR (bright region): {snr[100:150, 100:150].mean().item():.1f}")
print(f"Mean SNR (medium region): {snr[300:350, 300:350].mean().item():.1f}")
print(f"Mean SNR (dark region): {snr[400:450, 50:100].mean().item():.1f}")

## 7. Comparing Different Lens Configurations

Let's compare wide-angle, normal, and telephoto lenses.

In [ ]:
# Define different lens configurations
lens_configs = [
    {"focal_length": 24e-3, "f_number": 2.8, "name": "24mm Wide-angle"},
    {"focal_length": 50e-3, "f_number": 1.8, "name": "50mm Normal"},
    {"focal_length": 85e-3, "f_number": 1.8, "name": "85mm Portrait"},
    {"focal_length": 200e-3, "f_number": 2.8, "name": "200mm Telephoto"},
]

# Calculate properties for each lens
lens_properties = []
for lens in lens_configs:
    config = CameraConfig(
        focal_length=lens["focal_length"],
        f_number=lens["f_number"],
        object_distance=3.0,  # 3 meters
        sensor_size=(36e-3, 24e-3),  # Full frame
        wavelength=550e-9,
        n_pixels=512,
        pixel_size=6.5e-6,
    )
    camera = Camera(config)
    cam_info = camera.get_info()

    # Calculate properties
    props = {
        "name": lens["name"],
        "focal_length_mm": lens["focal_length"] * 1000,
        "f_number": lens["f_number"],
        "aperture_mm": lens["focal_length"] / lens["f_number"] * 1000,
        "magnification": abs(camera.calculate_magnification()),
        "image_distance_mm": camera.calculate_image_distance() * 1000,
        "resolution_um": camera.resolution_limit * 1e6,
        "fresnel": cam_info.get("fresnel_number", 0),
        "fov_deg": 2 * np.arctan(18e-3 / lens["focal_length"]) * 180 / np.pi,  # Horizontal FOV
    }

    # DOF
    near, far = camera.calculate_depth_of_field()
    props["dof_near"] = near
    props["dof_far"] = far
    props["dof_range"] = far - near

    # PSF
    props["psf"] = camera.compute_psf().cpu().numpy()

    lens_properties.append(props)

# Visualize comparison
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# PSF comparison
for idx, props in enumerate(lens_properties):
    ax = axes[0, idx]
    psf = props["psf"]
    center = psf.shape[0] // 2
    zoom = 30
    ax.imshow(psf[center - zoom : center + zoom, center - zoom : center + zoom], cmap="hot")
    ax.set_title(props["name"])
    ax.axis("off")

# Properties bar charts
names = [p["name"].split()[0] for p in lens_properties]  # Just focal length

# Aperture diameter
ax = axes[1, 0]
apertures = [p["aperture_mm"] for p in lens_properties]
bars = ax.bar(names, apertures, color="blue", alpha=0.7)
ax.set_title("Aperture Diameter")
ax.set_ylabel("Diameter (mm)")
ax.grid(True, axis="y", alpha=0.3)
for bar, val in zip(bars, apertures):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 1,
        f"{val:.1f}",
        ha="center",
        va="bottom",
    )

# Field of View
ax = axes[1, 1]
fovs = [p["fov_deg"] for p in lens_properties]
bars = ax.bar(names, fovs, color="green", alpha=0.7)
ax.set_title("Horizontal Field of View")
ax.set_ylabel("FOV (degrees)")
ax.grid(True, axis="y", alpha=0.3)
for bar, val in zip(bars, fovs):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 1,
        f"{val:.1f}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# Magnification
ax = axes[1, 2]
mags = [p["magnification"] for p in lens_properties]
bars = ax.bar(names, mags, color="orange", alpha=0.7)
ax.set_title("Magnification at 3m")
ax.set_ylabel("Magnification")
ax.grid(True, axis="y", alpha=0.3)
for bar, val in zip(bars, mags):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.001,
        f"{val:.3f}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# DOF Range
ax = axes[1, 3]
dof_ranges = [p["dof_range"] for p in lens_properties]
bars = ax.bar(names, dof_ranges, color="red", alpha=0.7)
ax.set_title("Depth of Field Range at 3m")
ax.set_ylabel("DOF Range (m)")
ax.grid(True, axis="y", alpha=0.3)
for bar, val in zip(bars, dof_ranges):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.05,
        f"{val:.2f}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# PSF cross-sections
ax = axes[2, 0]
colors = ["blue", "green", "orange", "red"]
for props, color in zip(lens_properties, colors):
    psf = props["psf"]
    center = psf.shape[0] // 2
    profile = psf[center, :]
    ax.plot(
        profile[center - 50 : center + 50], label=props["name"].split()[0], color=color, linewidth=2
    )
ax.set_title("PSF Cross-sections")
ax.set_xlabel("Pixels")
ax.set_ylabel("Intensity")
ax.legend()
ax.grid(True, alpha=0.3)

# Resolution comparison
ax = axes[2, 1]
resolutions = [p["resolution_um"] for p in lens_properties]
bars = ax.bar(names, resolutions, color="purple", alpha=0.7)
ax.set_title("Diffraction Limit")
ax.set_ylabel("Resolution (um)")
ax.grid(True, axis="y", alpha=0.3)
for bar, val in zip(bars, resolutions):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.05,
        f"{val:.1f}",
        ha="center",
        va="bottom",
    )

# Fresnel number
ax = axes[2, 2]
fresnels = [p["fresnel"] for p in lens_properties]
bars = ax.bar(names, fresnels, color="cyan", alpha=0.7)
ax.set_title("Fresnel Number")
ax.set_ylabel("F")
ax.axhline(y=0.1, color="red", linestyle="--", label="Far-field limit")
ax.grid(True, axis="y", alpha=0.3)
ax.legend()
ax.set_yscale("log")

# Summary table
ax = axes[2, 3]
ax.axis("tight")
ax.axis("off")
table_data = []
table_data.append(["Lens", "f/#", "FOV", "DOF(m)"])
for props in lens_properties:
    table_data.append(
        [
            props["name"].split()[0],
            f"{props['f_number']:.1f}",
            f"{props['fov_deg']:.1f}",
            f"{props['dof_range']:.2f}",
        ]
    )
table = ax.table(
    cellText=table_data, cellLoc="center", loc="center", colWidths=[0.3, 0.2, 0.2, 0.2]
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
# Style header row
for i in range(4):
    table[(0, i)].set_facecolor("#4CAF50")
    table[(0, i)].set_text_props(weight="bold", color="white")

plt.suptitle("Lens Comparison: Wide-angle to Telephoto", fontsize=14)
plt.tight_layout()
plt.show()

# Print detailed comparison
print("\nDetailed Lens Comparison at 3m Object Distance:")
print("=" * 90)
print(
    f"{'Lens':<15} {'Aperture':<12} {'FOV':<10} {'Magnif.':<10} {'DOF Near':<10} {'DOF Far':<10} {'Regime':<12}"
)
print("-" * 90)
for props in lens_properties:
    regime = "Far-field" if props["fresnel"] < 0.1 else "Near-field"
    print(
        f"{props['name']:<15} {props['aperture_mm']:>7.1f} mm  "
        f"{props['fov_deg']:>7.1f}    {props['magnification']:>8.4f}x  "
        f"{props['dof_near']:>8.2f}m  {props['dof_far']:>8.2f}m  {regime:<12}"
    )

## 8. Factory Function and Propagator Selection

The camera automatically selects the appropriate propagator based on the Fresnel number.

In [ ]:
# Test propagator selection for different scenarios
test_scenarios = [
    {"name": "Macro Photography", "focal_length": 100e-3, "object_distance": 0.3},
    {"name": "Portrait", "focal_length": 85e-3, "object_distance": 2.0},
    {"name": "Landscape", "focal_length": 24e-3, "object_distance": 100.0},
    {"name": "Astrophotography", "focal_length": 500e-3, "object_distance": float("inf")},
]

print("Propagator Selection Analysis:")
print("=" * 70)
print(
    f"{'Scenario':<20} {'Focal Length':<15} {'Object Dist':<15} {'Fresnel #':<12} {'Propagator':<15}"
)
print("-" * 70)

for scenario in test_scenarios:
    config = CameraConfig(
        focal_length=scenario["focal_length"],
        f_number=2.8,
        object_distance=scenario["object_distance"],
        wavelength=550e-9,
        n_pixels=256,
        pixel_size=6.5e-6,
    )

    camera = create_instrument(config)  # Using factory function
    cam_info = camera.get_info()
    fresnel = cam_info.get("fresnel_number", 0)

    # Determine propagator type (simplified check)
    if fresnel < 0.1 or scenario["object_distance"] == float("inf"):
        propagator = "Fraunhofer"
    else:
        propagator = "Angular Spectrum"

    obj_dist_str = (
        "inf"
        if scenario["object_distance"] == float("inf")
        else f"{scenario['object_distance']:.1f}m"
    )

    print(
        f"{scenario['name']:<20} {scenario['focal_length'] * 1000:>10.0f} mm  "
        f"{obj_dist_str:>14}  {fresnel:>11.4f}  {propagator:<15}"
    )

print("\nNote: Fresnel number F = a^2/(lambda*z) where a is aperture radius, z is distance")
print("      F < 0.1: Far-field (Fraunhofer)")
print("      F >= 0.1: Near-field (Angular Spectrum)")

# Verify camera is created correctly through factory
print("\n" + "=" * 50)
print("Factory Function Verification:")
print("=" * 50)

config = CameraConfig(
    focal_length=35e-3,
    f_number=1.8,
    sensor_size=(36e-3, 24e-3),
    object_distance=1.5,
    wavelength=550e-9,
    n_pixels=512,
    pixel_size=6.5e-6,
)

camera = create_instrument(config)

print(f"Instrument type: {camera.get_instrument_type()}")
print(f"Focal length: {camera.focal_length * 1000:.0f}mm")
print(f"f-number: f/{camera.f_number}")
print(f"Resolution limit: {camera.resolution_limit * 1e6:.1f}um")
print(f"Magnification: {camera.calculate_magnification():.4f}x")
near, far = camera.calculate_depth_of_field()
print(f"DOF: {near:.2f}m to {far:.2f}m")
print("\nCamera successfully created via factory function!")

## Summary

This notebook demonstrated:

1. **Camera Configuration**: Setting up various camera systems with different focal lengths and apertures
2. **PSF Analysis**: Computing point spread functions with defocus effects
3. **Depth of Field**: Visualizing DOF dependence on aperture size
4. **Image Formation**: Simulating actual imaging through camera optics
5. **Noise Modeling**: Adding realistic sensor noise (shot, read, dark current)
6. **Lens Comparison**: Comparing wide-angle to telephoto characteristics
7. **Propagator Selection**: Automatic selection based on Fresnel number
8. **Factory Pattern**: Using the unified instrument creation interface

Key takeaways:
- Smaller f-numbers (larger apertures) provide shallower DOF but better light gathering
- Fresnel number determines the propagation regime (far-field vs. near-field)
- Longer focal lengths provide higher magnification but narrower field of view
- Diffraction limit scales with f-number: spot size ∝ λ × f/#
- The factory pattern ensures consistent instrument creation across all types